# NetCDF handling

NetCDF formatted files are much faster to read and write for large datasets. In order to make the most of this, both the `ScmDataFrame` and `ScmRun` objects have the ability to read and write netCDF files.

In [1]:
import traceback
from glob import glob

import numpy as np
import xarray as xr

from scmdata.run import ScmRun, df_append
from scmdata.netcdf import nc_to_run

<IPython.core.display.Javascript object>

pyam - INFO: Running in a notebook, setting `pyam` logging level to `logging.INFO` and adding stderr handler


In [2]:
OUT_FNAME = "/tmp/out_runs.nc"

In [3]:
def new_timeseries(
    n=100,
    count=1,
    model="example",
    scenario="ssp119",
    variable="Surface Temperature",
    cls=ScmRun,
    **kwargs,
):
    data = np.random.rand(n, count) * np.arange(n)[:, np.newaxis]
    index = 2000 + np.arange(n)
    return cls(
        data,
        columns={
            "model": model,
            "scenario": scenario,
            "variable": variable,
            "region": "World",
            "unit": "K",
            **kwargs,
        },
        index=index,
    )

Let's create an `ScmRun` which contains a few variables and a number of runs. Such a dataframe would be used to store the results from an ensemble of simple climate model runs.

In [4]:
# NBVAL_IGNORE_OUTPUT
runs = df_append(
    [
        new_timeseries(
            count=3,
            variable=[
                "Surface Temperature",
                "Atmospheric Concentrations|CO2",
                "Radiative Forcing",
            ],
            run_id=run_id,
        )
        for run_id in range(10)
    ]
)
runs

<scmdata.ScmRun (timeseries: 30, timepoints: 100)>
Time:
	Start: 2000-01-01T00:00:00
	End: 2099-01-01T00:00:00
Meta:
	      model scenario                        variable region unit  run_id
	0   example   ssp119             Surface Temperature  World    K       0
	1   example   ssp119  Atmospheric Concentrations|CO2  World    K       0
	2   example   ssp119               Radiative Forcing  World    K       0
	3   example   ssp119             Surface Temperature  World    K       1
	4   example   ssp119  Atmospheric Concentrations|CO2  World    K       1
	5   example   ssp119               Radiative Forcing  World    K       1
	6   example   ssp119             Surface Temperature  World    K       2
	7   example   ssp119  Atmospheric Concentrations|CO2  World    K       2
	8   example   ssp119               Radiative Forcing  World    K       2
	9   example   ssp119             Surface Temperature  World    K       3
	10  example   ssp119  Atmospheric Concentrations|CO2  World    K    

## Reading/Writing to NetCDF4

Writing the runs to disk is easy. The one trick is that each variable and dimension combination must have unique metadata. If they do not, you will receive an error message like the below.

In [5]:
try:
    runs.to_nc(OUT_FNAME, dimensions=["region"])
except ValueError:
    traceback.print_exc(limit=0, chain=False)

Traceback (most recent call last):
ValueError: region dimension is not unique for variable Atmospheric Concentrations|CO2


In our dataset, there is more than one "run_id" per variable hence we need to add an additional dimension: `run_id`.

In [6]:
runs.to_nc(OUT_FNAME, dimensions=["run_id"])

The output netCDF file can be read using the `from_nc` method, `nc_to_run` function or directly using `xarray`.

In [7]:
# NBVAL_IGNORE_OUTPUT
ScmRun.from_nc(OUT_FNAME)

<scmdata.ScmRun (timeseries: 30, timepoints: 100)>
Time:
	Start: 2000-01-01T00:00:00
	End: 2099-01-01T00:00:00
Meta:
	    run_id                        variable region scenario unit    model
	30       0  Atmospheric Concentrations|Co2  World   ssp119    K  example
	31       1  Atmospheric Concentrations|Co2  World   ssp119    K  example
	32       2  Atmospheric Concentrations|Co2  World   ssp119    K  example
	33       3  Atmospheric Concentrations|Co2  World   ssp119    K  example
	34       4  Atmospheric Concentrations|Co2  World   ssp119    K  example
	35       5  Atmospheric Concentrations|Co2  World   ssp119    K  example
	36       6  Atmospheric Concentrations|Co2  World   ssp119    K  example
	37       7  Atmospheric Concentrations|Co2  World   ssp119    K  example
	38       8  Atmospheric Concentrations|Co2  World   ssp119    K  example
	39       9  Atmospheric Concentrations|Co2  World   ssp119    K  example
	40       0               Radiative Forcing  World   ssp119    K  exa

In [8]:
# NBVAL_IGNORE_OUTPUT
nc_to_run(ScmRun, OUT_FNAME)

<scmdata.ScmRun (timeseries: 30, timepoints: 100)>
Time:
	Start: 2000-01-01T00:00:00
	End: 2099-01-01T00:00:00
Meta:
	    run_id                        variable region scenario unit    model
	60       0  Atmospheric Concentrations|Co2  World   ssp119    K  example
	61       1  Atmospheric Concentrations|Co2  World   ssp119    K  example
	62       2  Atmospheric Concentrations|Co2  World   ssp119    K  example
	63       3  Atmospheric Concentrations|Co2  World   ssp119    K  example
	64       4  Atmospheric Concentrations|Co2  World   ssp119    K  example
	65       5  Atmospheric Concentrations|Co2  World   ssp119    K  example
	66       6  Atmospheric Concentrations|Co2  World   ssp119    K  example
	67       7  Atmospheric Concentrations|Co2  World   ssp119    K  example
	68       8  Atmospheric Concentrations|Co2  World   ssp119    K  example
	69       9  Atmospheric Concentrations|Co2  World   ssp119    K  example
	70       0               Radiative Forcing  World   ssp119    K  exa

In [9]:
# NBVAL_IGNORE_OUTPUT
xr.load_dataset(OUT_FNAME)

<xarray.Dataset>
Dimensions:                          (run_id: 10, time: 100)
Coordinates:
  * time                             (time) float64 9.467e+08 ... 4.071e+09
  * run_id                           (run_id) int64 0 1 2 3 4 5 6 7 8 9
Data variables:
    atmospheric_concentrations__co2  (run_id, time) float64 0.0 0.8594 ... 52.18
    radiative_forcing                (run_id, time) float64 0.0 0.8235 ... 76.51
    surface_temperature              (run_id, time) float64 0.0 0.4 ... 21.86
Attributes:
    created_at:        2020-04-09T11:19:52.048940
    _scmdata_version:  0.4.0+51.gf720903.dirty

Sometimes if you have complicated ensemble runs it might be more efficient to split the data into smaller subsets.

In the below example we iterate over scenarios to produce a netCDF file per scenario.

In [13]:
large_run = []

# 10 runs for each scenario
for sce in ["ssp119", "ssp370", "ssp585"]:
    large_run.extend(
        [
            new_timeseries(
                count=3,
                scenario=sce,
                variable=[
                    "Surface Temperature",
                    "Atmospheric Concentrations|CO2",
                    "Radiative Forcing",
                ],
                paraset_id=paraset_id,
            )
            for paraset_id in range(10)
        ]
    )

large_run = df_append(large_run)

# also set a run_id (often we'd have paraset_id and run_id,
# one which keeps track of the parameter set we've run and
# the other which keeps track of the run in a large ensemble
# even though they don't really overlap properly)
large_run["run_id"] = large_run.meta.index.values
large_run

<scmdata.ScmRun (timeseries: 90, timepoints: 100)>
Time:
	Start: 2000-01-01T00:00:00
	End: 2099-01-01T00:00:00
Meta:
	       model scenario                        variable region unit  paraset_id  \
	180  example   ssp119             Surface Temperature  World    K           0   
	181  example   ssp119  Atmospheric Concentrations|CO2  World    K           0   
	182  example   ssp119               Radiative Forcing  World    K           0   
	183  example   ssp119             Surface Temperature  World    K           1   
	184  example   ssp119  Atmospheric Concentrations|CO2  World    K           1   
	..       ...      ...                             ...    ...  ...         ...   
	265  example   ssp585  Atmospheric Concentrations|CO2  World    K           8   
	266  example   ssp585               Radiative Forcing  World    K           8   
	267  example   ssp585             Surface Temperature  World    K           9   
	268  example   ssp585  Atmospheric Concentrations|CO2  World  

The one problem with this approach is that you get very sparse arrays because the data is written on a 100 x 30 x 90 (time points x paraset_id x run_id) grid but there's only 90 timeseries so you end up with 180 timeseries worth of nans. An obvious solution is for the user to drop the run_id meta before saving (because it's redundant) but not sure that realisation can be automated...

In [11]:
for sce_df in large_run.groupby("scenario"):
    sce = sce_df.get_unique_meta("scenario", True)
    sce_df.to_nc("/tmp/out-{}.nc".format(sce), dimensions=["run_id", "paraset_id"])

My update above appears to have broken the below...

Data for each scenario can then be loaded independently instead of having to load all the data and then filtering

In [12]:
# NBVAL_IGNORE_OUTPUT
ScmRun.from_nc("/tmp/out-ssp585.nc").filter("Surface Temperature").line_plot()

Failed reading netdf file: /tmp/out-ssp585.nc
Traceback (most recent call last):
  File "/Users/znicholls/Documents/AGCEC/MCastle/scmdata/src/scmdata/netcdf.py", line 203, in nc_to_run
    return _read_nc(cls, ds)
  File "/Users/znicholls/Documents/AGCEC/MCastle/scmdata/src/scmdata/netcdf.py", line 149, in _read_nc
    if not valid_mask[it.multi_index]:
  File "/Users/znicholls/Documents/AGCEC/MCastle/scmdata/venv/lib/python3.7/site-packages/numpy/ma/core.py", line 3188, in __getitem__
    dout = self.data[indx]
IndexError: too many indices for array


AttributeError: 'NoneType' object has no attribute 'filter'

In [ ]:
# NBVAL_IGNORE_OUTPUT
# Load all scenarios
df_append([ScmRun.from_nc(fname) for fname in glob("/tmp/out-*.nc")])